In [1]:
import ee
import folium
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# This is needed in python to add layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=hSByOYt2H5MF6LGOh73YAJeP3qMEY7MVE7DJ6VfgDsQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh6x3Yf1qt2dJKiJllR2UaJe12T0QLC84m9gIMMIFugf4pF6i9hOik

Successfully saved authorization token.


In [7]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

lat = 41.0340
lon = -73.7629

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# Define a map centered on White Plains, NY.
map_bare = folium.Map(location=[lat, lon], zoom_start=8)

# Add the masked image layer to the map and display it.
map_bare.add_ee_layer(bare.selfMask(), None, 'bare')
display(map_bare)

In [8]:
# Load a 2012 nightlights image.
nl_2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl_2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Define a map centered on White Plains.
map_zones = folium.Map(location=[lat, lon], zoom_start=8)

# Display the thresholded image as three distinct zones near White Plains.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
map_zones.add_ee_layer(
    zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones')
display(map_zones)